In [2]:
import azureml.core
from azureml.core import Workspace
print(azureml.core.VERSION)

1.0.23


#### If workspace set up is needed

In [ ]:
ws = Workspace.create(name='mercurial1',
                      subscription_id='f6058e84-f9c7-4e6b-a169-6b081a038f56',
                      resource_group='SeattleProofing',
                      create_resource_group=False,
                      location='WestCentralUS' 
                     )

In [ ]:
ws.write_config()

#### If workspace is already set up

In [3]:
ws = Workspace.from_config()

Found the config file in: C:\Users\casocha\Documents\Kaggle\.azureml\config.json


In [4]:
ws.get_details()

{'id': '/subscriptions/f6058e84-f9c7-4e6b-a169-6b081a038f56/resourceGroups/SeattleProofing/providers/Microsoft.MachineLearningServices/workspaces/mercurial1',
 'name': 'mercurial1',
 'location': 'westcentralus',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'workspaceid': 'c80865c0-9f3c-40c4-ad61-cc09d2f7e7aa',
 'description': '',
 'friendlyName': 'mercurial1',
 'creationTime': '2019-04-09T22:42:49.9898063+00:00',
 'containerRegistry': '/subscriptions/f6058e84-f9c7-4e6b-a169-6b081a038f56/resourcegroups/seattleproofing/providers/microsoft.containerregistry/registries/mercuriaacrvuuidymc',
 'keyVault': '/subscriptions/f6058e84-f9c7-4e6b-a169-6b081a038f56/resourcegroups/seattleproofing/providers/microsoft.keyvault/vaults/mercuriakeyvaultkbnibtju',
 'applicationInsights': '/subscriptions/f6058e84-f9c7-4e6b-a169-6b081a038f56/resourcegroups/seattleproofing/providers/microsoft.insights/components/mercuriainsightsoayhykyq',
 'identityPrincipalId': '5ff1f39c-660b-4e13-82e2-cb4a44910

#### Create experiment for tracking

In [5]:
experiment_name = 'test'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

#### Load data locally

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


%matplotlib inline
sns.set_style('whitegrid')

In [7]:
df_train = pd.read_csv(r'C:\Users\casocha\Downloads\dont-overfit-ii/train.csv',index_col = 'id')
df_train.head()

,target,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
id,,,,,,,,,,,,,,,,,,,,,
0,1.0,-0.098,2.165,0.681,-0.614,1.309,-0.455,-0.236,0.276,-2.246,...,0.867,1.347,0.504,-0.649,0.672,-2.097,1.051,-0.414,1.038,-1.065
1,0.0,1.081,-0.973,-0.383,0.326,-0.428,0.317,1.172,0.352,0.004,...,-0.165,-1.695,-1.257,1.359,-0.808,-1.624,-0.458,-1.099,-0.936,0.973
2,1.0,-0.523,-0.089,-0.348,0.148,-0.022,0.404,-0.023,-0.172,0.137,...,0.013,0.263,-1.222,0.726,1.444,-1.165,-1.544,0.004,0.800,-1.211
3,1.0,0.067,-0.021,0.392,-1.637,-0.446,-0.725,-1.035,0.834,0.503,...,-0.404,0.640,-0.595,-0.966,0.900,0.467,-0.562,-0.254,-0.533,0.238
4,1.0,2.347,-0.831,0.511,-0.021,1.225,1.594,0.585,1.509,-0.012,...,0.898,0.134,2.415,-0.996,-1.006,1.378,1.246,1.478,0.428,0.253


In [8]:
df_test = pd.read_csv(r'C:\Users\casocha\Downloads\dont-overfit-ii/test.csv',index_col = 'id')
df_test.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
id,,,,,,,,,,,,,,,,,,,,,
250,0.500,-1.033,-1.595,0.309,-0.714,0.502,0.535,-0.129,-0.687,1.291,...,-0.088,-2.628,-0.845,2.078,-0.277,2.132,0.609,-0.104,0.312,0.979
251,0.776,0.914,-0.494,1.347,-0.867,0.480,0.578,-0.313,0.203,1.356,...,-0.683,-0.066,0.025,0.606,-0.353,-1.133,-3.138,0.281,-0.625,-0.761
252,1.750,0.509,-0.057,0.835,-0.476,1.428,-0.701,-2.009,-1.378,0.167,...,-0.094,0.351,-0.607,-0.737,-0.031,0.701,0.976,0.135,-1.327,2.463
253,-0.556,-1.855,-0.682,0.578,1.592,0.512,-1.419,0.722,0.511,0.567,...,-0.336,-0.787,0.255,-0.031,-0.836,0.916,2.411,1.053,-1.601,-1.529
254,0.754,-0.245,1.173,-1.623,0.009,0.370,0.781,-1.763,-1.432,-0.930,...,2.184,-1.090,0.216,1.186,-0.143,0.322,-0.068,-0.156,-1.153,0.825


In [9]:
y_train = df_train['target']
df_train = df_train.drop(['target'], axis=1)

In [10]:
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
id,,,,,,,,,,,,,,,,,,,,,
0,-0.098,2.165,0.681,-0.614,1.309,-0.455,-0.236,0.276,-2.246,1.825,...,0.867,1.347,0.504,-0.649,0.672,-2.097,1.051,-0.414,1.038,-1.065
1,1.081,-0.973,-0.383,0.326,-0.428,0.317,1.172,0.352,0.004,-0.291,...,-0.165,-1.695,-1.257,1.359,-0.808,-1.624,-0.458,-1.099,-0.936,0.973
2,-0.523,-0.089,-0.348,0.148,-0.022,0.404,-0.023,-0.172,0.137,0.183,...,0.013,0.263,-1.222,0.726,1.444,-1.165,-1.544,0.004,0.800,-1.211
3,0.067,-0.021,0.392,-1.637,-0.446,-0.725,-1.035,0.834,0.503,0.274,...,-0.404,0.640,-0.595,-0.966,0.900,0.467,-0.562,-0.254,-0.533,0.238
4,2.347,-0.831,0.511,-0.021,1.225,1.594,0.585,1.509,-0.012,2.198,...,0.898,0.134,2.415,-0.996,-1.006,1.378,1.246,1.478,0.428,0.253


#### Load data to remote cluster

In [11]:
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

AzureBlob mercuriastoragefzmxacta azureml-blobstore-c80865c0-9f3c-40c4-ad61-cc09d2f7e7aa


In [12]:
import azureml.data
from azureml.data.azure_storage_datastore import AzureFileDatastore, AzureBlobDatastore

ds.upload(src_dir=r'C:\Users\casocha\Downloads\dont-overfit-ii',
          target_path='test',
          overwrite=True,
          show_progress=True)

Uploading C:\Users\casocha\Downloads\dont-overfit-ii\sample_submission.csv
Uploading C:\Users\casocha\Downloads\dont-overfit-ii\test.csv
Uploading C:\Users\casocha\Downloads\dont-overfit-ii\train.csv
Uploaded C:\Users\casocha\Downloads\dont-overfit-ii\sample_submission.csv, 1 files out of an estimated total of 3
Uploaded C:\Users\casocha\Downloads\dont-overfit-ii\train.csv, 2 files out of an estimated total of 3
Uploaded C:\Users\casocha\Downloads\dont-overfit-ii\test.csv, 3 files out of an estimated total of 3


$AZUREML_DATAREFERENCE_9a89cf6399f0463bb8786dab713f3a6c

#### Create script to run remotely

In [13]:
import os
script_folder  = os.path.join(os.getcwd(), "beta_test")
os.makedirs(script_folder, exist_ok=True)

In [14]:
%%writefile $script_folder/train.py

import argparse
import os
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split

from azureml.core import Run

# let user feed in 2 parameters, the location of the data files (from datastore), and the regularization rate of the logistic regression model
parser = argparse.ArgumentParser()
parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder mounting point')
parser.add_argument('--regularization', type=float, dest='reg', default=0.01, help='regularization rate')
args = parser.parse_args()

data_folder = args.data_folder
print('Data folder:', data_folder)

# load train and test set into numpy arrays
# note we scale the pixel intensity values to 0-1 (by dividing it with 255.0) so the model can converge faster.
df_train = pd.read_csv(os.path.join(data_folder, 'train.csv'),index_col = 'id')
X_predict = pd.read_csv(os.path.join(data_folder, 'test.csv'),index_col = 'id')
y_train = df_train['target']
df_train = df_train.drop(['target'], axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(df_train, y_train, test_size=0.1, random_state=0)

print(X_train.shape, y_train.shape, X_valid.shape, y_valid.shape, sep = '\n')

# get hold of the current run
run = Run.get_context()

print('Train a logistic regression model with regularization rate of', args.reg)
clf = LogisticRegression(C=1.0/args.reg, random_state=42)
clf.fit(X_train, y_train)

print('Predict the test set')
y_hat = clf.predict(X_valid)

# calculate accuracy on the prediction
acc = np.average(y_hat == y_valid)
print('Accuracy is', acc)

run.log('regularization rate', np.float(args.reg))
run.log('accuracy', np.float(acc))

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=clf, filename='outputs/sklearn_mnist_model.pkl')

Overwriting C:\Users\casocha\Documents\Kaggle\beta_test/train.py


#### Create remote compute for training

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "cpucluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

In [15]:
compute_target = ws.compute_targets['cpucluster']

#### Create an estimator & run experiment

In [16]:
from azureml.train.estimator import Estimator

script_params = {
    '--data-folder': ds.path('beta_test').as_mount(),
    '--regularization': 0.8
}

est = Estimator(source_directory=script_folder,
                script_params=script_params,
                compute_target=compute_target,
                entry_script='train.py',
                conda_packages=['scikit-learn'])

In [17]:
run = exp.submit(config=est)
run

Experiment,Id,Type,Status,Details Page,Docs Page
test,test_1554933763_caaa1c6b,azureml.scriptrun,Queued,Link to Azure Portal,Link to Documentation
